In [1]:
import time
import torch
import datasets
import evaluate
import numpy as np
from transformers import (
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    Trainer,
    TrainingArguments,
    DataCollatorForSeq2Seq,
)
from peft import LoraConfig, get_peft_model, TaskType

# بارگذاری dataset
dataset = datasets.load_dataset("knkarthick/dialogsum")
dataset

/home/ai/Desktop/Fine-Tune/.env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
'(ProtocolError('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer')), '(Request ID: 01e67729-6eaf-41a0-b610-1149360d03c2)')' thrown while requesting HEAD https://huggingface.co/datasets/knkarthick/dialogsum/resolve/main/README.md
Retrying in 1s [Retry 1/5].
Using the latest cached version of the dataset since knkarthick/dialogsum couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'default' at /home/ai/.cache/huggingface/datasets/knkarthick___dialogsum/default/0.0.0/a968e7aee0602e257935f1321a02e4287f7d5848 (last modified on Wed Oct  1 16:22:48 2025).


DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 12460
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 500
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 1500
    })
})

In [2]:
# بارگذاری مدل و tokenizer
model_name = 'google/flan-t5-base'
origin_model = AutoModelForSeq2SeqLM.from_pretrained(
    model_name,
    torch_dtype=torch.float32  # یا torch.float16 برای کاهش مصرف حافظه
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# تنظیمات مهم برای training
origin_model.config.use_cache = False  # برای gradient checkpointing

`torch_dtype` is deprecated! Use `dtype` instead!


In [3]:
def tokenize_function(example):
    """
    تابع tokenize اصلاح شده - مشکل اصلی اینجا بود
    """
    start_prompt = "Summarize the following conversation.\n\n"
    end_prompt = "\n\nSummary: "
    
    # ساخت prompts
    prompts = [start_prompt + dialogue + end_prompt for dialogue in example["dialogue"]]
    
    # Tokenize inputs
    model_inputs = tokenizer(
        prompts,
        padding="max_length",
        truncation=True,
        max_length=512,
    )
    
    # Tokenize targets با استفاده از target tokenizer context
    # این بخش برای T5 مهمه
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            example["summary"],
            padding="max_length",
            truncation=True,
            max_length=128,
        )
    
    # تبدیل padding tokens به -100
    labels_ids = []
    for label in labels["input_ids"]:
        label_ids = [(l if l != tokenizer.pad_token_id else -100) for l in label]
        labels_ids.append(label_ids)
    
    model_inputs["labels"] = labels_ids
    
    return model_inputs

# اعمال tokenization
ds_tokenized = dataset.map(tokenize_function, batched=True)
ds_tokenized = ds_tokenized.remove_columns(["id", "dialogue", "summary", "topic"])

Map:   0%|          | 0/12460 [00:00<?, ? examples/s]/home/ai/Desktop/Fine-Tune/.env/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:4007: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Map: 100%|██████████| 1500/1500 [00:00<00:00, 2211.10 examples/s]


In [4]:
# تنظیمات LoRA
lora_config = LoraConfig(
    task_type=TaskType.SEQ_2_SEQ_LM,
    r=32,  # می‌تونید کم کنید برای کاهش مصرف حافظه
    lora_alpha=32,  # معمولاً برابر یا 2 برابر r
    target_modules=['q', 'v'],  # برای T5
    lora_dropout=0.05,
    bias="none",
)

# اعمال LoRA به مدل
lora_model = get_peft_model(origin_model, lora_config)

In [6]:
# نمایش تعداد پارامترها
def print_trainable_parameters(model):
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params:,} || "
        f"all params: {all_param:,} || "
        f"trainable: {100 * trainable_params / all_param:.2f}%"
    )

print_trainable_parameters(lora_model)

trainable params: 3,538,944 || all params: 251,116,800 || trainable: 1.41%


In [7]:
# Data Collator - مهم برای padding صحیح
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=lora_model,
    label_pad_token_id=-100,  # این مهمه!
    pad_to_multiple_of=8  # برای بهینه‌سازی سرعت
)

In [14]:
# تنظیمات training اصلاح شده
output_dir = f'./saved_models/flan-t5-lora-{str(int(time.time()))}'

training_args = TrainingArguments(
    output_dir=output_dir,
    learning_rate=3e-4,  # برای LoRA معمولاً learning rate بالاتر
    num_train_epochs=3,  # یا می‌تونید max_steps استفاده کنید
    per_device_train_batch_size=8,  # بسته به RAM GPU
    per_device_eval_batch_size=8,
    warmup_ratio=0.1,  # 10% از steps برای warmup
    weight_decay=0.01,
    logging_steps=10,
    eval_strategy="steps",
    eval_steps=50,
    save_strategy="steps",
    save_steps=100,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    fp16=torch.cuda.is_available(),  # اگر GPU دارید
    gradient_checkpointing=True,  # برای کاهش مصرف حافظه
    gradient_accumulation_steps=4,  # برای batch size مؤثر بزرگتر
    report_to="none",  # یا "tensorboard" اگر نصب دارید
)

# ساخت trainer
trainer = Trainer(
    model=lora_model,
    args=training_args,
    train_dataset=ds_tokenized["train"],
    eval_dataset=ds_tokenized["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

/tmp/ipykernel_17241/2341123853.py:27: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [15]:
# شروع training
print("شروع آموزش...")
trainer.train()

شروع آموزش...


/home/ai/Desktop/Fine-Tune/.env/lib/python3.12/site-packages/torch/utils/checkpoint.py:85: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

In [ ]:
# ذخیره مدل نهایی
trainer.save_model()
lora_model.save_pretrained(output_dir)

print(f"مدل در {output_dir} ذخیره شد.")

In [ ]:
# تست مدل آموزش دیده
def test_model(model, tokenizer, dialogue):
    prompt = f"Summarize the following conversation.\n\n{dialogue}\n\nSummary: "

    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512)

    with torch.no_grad():
        outputs = model.generate(
            **inputs, max_new_tokens=128, num_beams=4, temperature=0.7, do_sample=False
        )

    summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return summary

In [ ]:
# تست با یک نمونه
test_dialogue = dataset["test"][0]["dialogue"]
test_summary = test_model(lora_model, tokenizer, test_dialogue)
print(f"\nDialogue: {test_dialogue[:200]}...")
print(f"\nGenerated Summary: {test_summary}")
print(f"\nOriginal Summary: {dataset['test'][0]['summary']}")